# 位置参照情報ダウンロードサービスのCSV をGeoJSON に変換するサンプル

## 位置参照情報のダウンロードページ

https://nlftp.mlit.go.jp/cgi-bin/isj/dls/_choose_method.cgi

「街区レベル位置参照情報」と「大字・町丁目レベル位置参照情報」が入手可能で、それぞれファイルの定義が異なります。

## a) 大字・町丁目レベル位置参照情報のデータ

### a-1) ZIP ファイルの中身

ZIPファイル名：'XX000-17.0b.zip' (XXは都道府県コード)

<pre>
XX000-17.0b.zip
    |-XX000-17.0b
        |-XX_2023.CSV
        |-format.2023.html
        |-META_XX_2023.xml
</pre>
ZIPファイル内は'XX000-12.0b' フォルダ内に3つのファイルが含まれる。そのうち 'XX_2023.CSV' のファイルが大字・町丁目のCSVデータ

### a-2) ファイルの定義

ファイル形式はCSV(Comma Separated Values)、文字コードは、Shift-JISです。
 
     項目                        定義                                                                    記述例  
    ----------------------------------------------------------------------------------------------------------------------------  
    都道府県コード              JIS都道府県コード                                                       "13"  
    都道府県名                  当該範囲の都道府県名                                                    "東京都"  
    市区町村コード              JIS市区町村コード                                                       "13101"  
    市区町村名                  当該範囲の市区町村名 （郡部は郡名，政令指定都市の区名も含む）           "千代田区"  
    大字町丁目コード	        大字町丁目コード（JIS市区町村コード＋独自7桁）                          "131010005002"  
    大字町丁目名                当該範囲の大字・町丁目名    （町丁目の数字は漢数字）                    "霞が関二丁目"  
    緯度                        十進経緯度（単位：度、小数点以下第6位まで、半角）                       "35.675706"  
    経度                        十進経緯度（単位：度、小数点以下第6位まで、半角）                       "139.750734"  
    原典資料コード              大字・町丁目位置参照情報作成における原典資料を表すコード                "1"  
                               (1:自治体資料、2:街区レベル位置参照情報、3:1/25000地形図、0:その他資料)   
    大字・字・丁目区分コード    大字／字／丁目の区別を表すコード	                                    "3"
                               (1:大字、2:字、3:丁目、0:不明（通称）)  
    ----------------------------------------------------------------------------------------------------------------------------  

## b) 街区レベル位置参照情報のデータ
### b-1) ZIP ファイルの中身

ZIPファイル名：'XX000-17.0b.zip' (XXは都道府県コード)

<pre>
XX000-22.0a.zip
    |-XX000-22.0a
        |-XX_2023.CSV
        |-format.2023.html
        |-md_XX_2023.xml
</pre>
ZIPファイル内は'XX000-12.0b' フォルダ内に3つのファイルが含まれる。そのうち 'XX_2023.CSV' のファイルが大字・町丁目のCSVデータ

### b-2) ファイルの定義

ファイル形式はCSV(Comma Separated Values)、文字コードは、Shift-JISです。

    項目                備考                                                           例  
    ----------------------------------------------------------------------------------------------------------------------------  
    都道府県名          当該範囲の都道府県名                                           "福島県"  
    市区町村名          当該範囲の市区町村名（郡部は郡名、政令指定都市の区名も含む）   "本宮市"  
    大字・丁目名        当該範囲の大字・丁目名（丁目の数字は漢数字）	               "岩根"  
    小字・通称名        当該範囲の小字・通称名                                         "下桶"  
    街区符号・地番      当該範囲の街区符号・地番                                       "126"  
                       （通常は1以上の半角整数。ただし地域により漢字・アルファベット等もある）
    座標系番号          平面直角座標系の座標系番号（1～19までの半角整数）              "9"  
    Ｘ座標              平面直角座標系の座標系原点からの距離                           "164065.1"  
                          メートル単位、北方向プラス（小数第1位まで、半角）   
    Ｙ座標              平面直角座標系の座標系原点からの距離                           "43616.6"  
                          メートル単位、東方向プラス（小数第1位まで、半角）  
    緯度                十進経緯度（小数点以下第6位まで、半角）                        "37.477545"  
    経度                十進経緯度（小数点以下第6位まで、半角）                        "140.326492"  
    住居表示フラグ      1：住居表示実施、0：住居表示未実施（半角）                     "0"  
    代表フラグ          1：代表する、0：代表しない（半角）                             "1"  
                        1つの街区符号または地番が複数の代表点に対応付けられる場合などに、
                        そのうちの1つに便宜的に代表フラグを立てる。
                        1つの街区符号または地番に対し1つのデータの場合は、当該データに対して代表フラグを立てる
    更新前履歴フラグ    1：新規作成、2：名称変更、3：削除、0：変更なし（半角）	     "0"  
                        （前年度対応内容）
    更新後履歴フラグ    1：新規作成、2：名称変更、3：削除、0：変更なし（半角）	     "1"  
                        （今年度対応内容）
    ----------------------------------------------------------------------------------------------------------------------------  

## プログラム

処理する方式は、「街区レベル位置参照情報」と「大字・町丁目レベル位置参照情報」で同じですが、ファイルの定義が異なるため、それぞれを関数として定義。

In [1]:
import csv
import json
import copy

def ooaza_cho_csv_to_geojson(csv_file_path :str, json_file_path :str):
    """
    大字・町丁目のCSVデータをcsv.DictReaderで読み込み
    GeoJSON のdictに反映してjsonとして保存
    """
    # FeatureCollection を dict として定義
    feature_collection = {
        "type": "FeatureCollection",
        "crs": {"type": "name", "properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}},
        "features": []
    }
    # feature (point) を dict として定義
    feature_template = {
        "type": "Feature",
        "id": 0,
        "geometry": {
            "type": "Point",
            "coordinates": [0, 0]
        },
        "properties": {
            "OID": 0,
            "都道府県コード": "",
            "都道府県名": "",
            "市区町村コード": "",
            "市区町村名": "",
            "大字町丁目コード": "",
            "大字町丁目名": "",
            "緯度": 0,
            "経度": 0,
            "原典資料コード": 0,
            "大字_字_丁目区分コード": 0
        }
    }
    #csv.DictReaderで読込し、feature のテンプレートに適用後、FeatureCollectionにアペンド
    with open(csv_file_path, "r", encoding='shift_jis') as f:
        reader = csv.DictReader(f)
        fcnt = 0
        for row in reader:
            #feature のテンプレートに適用する処理
            feature = copy.deepcopy(feature_template)
            feature["id"] = fcnt
            feature["geometry"]["coordinates"] = [float(row["経度"]), float(row["緯度"])]
            feature["properties"]["OID"] = fcnt
            feature["properties"]["都道府県コード"] = row["都道府県コード"]
            feature["properties"]["都道府県名"] = row["都道府県名"]
            feature["properties"]["市区町村コード"] = row["市区町村コード"]
            feature["properties"]["市区町村名"] = row["市区町村名"]
            feature["properties"]["大字町丁目コード"] = row["大字町丁目コード"]
            feature["properties"]["大字町丁目名"] = row["大字町丁目名"]
            feature["properties"]["緯度"] = float(row["緯度"])
            feature["properties"]["経度"] = float(row["経度"])
            feature["properties"]["原典資料コード"] = int(row["原典資料コード"])
            feature["properties"]["大字_字_丁目区分コード"] = int(row["大字・字・丁目区分コード"])
            #FeatureCollectionにアペンド
            feature_collection["features"].append(feature)
            #次のレコード用にidをインクリメント
            fcnt +=1
    #FeatureCollectionをjsonとして書き出し保存
    with open(json_file_path, "w", encoding="utf_8") as f:
        json.dump(feature_collection, f, ensure_ascii=False, indent=4)
        
    return fcnt


def gaiku_csv_to_geojson(csv_file_path :str, json_file_path :str):
    """
    街区のCSVデータをcsv.DictReaderで読み込み
    GeoJSON のdictに反映してjsonとして保存
    """
    # FeatureCollection を dict として定義
    feature_collection = {
        "type": "FeatureCollection",
        "crs": {"type": "name", "properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"}},
        "features": []
    }
    # feature (point) を dict として定義
    feature_template = {
        "type": "Feature",
        "id": 0,
        "geometry": {
            "type": "Point",
            "coordinates": [0, 0]
        },
        "properties": {
            "OID": 0,
            "都道府県名": "",
            "市区町村名": "",
            "大字_丁目名": "",
            "小字_通称名": "",
            "街区符号_地番": "",
            "座標系番号": 0,
            "Ｘ座標": 0,
            "Ｙ座標": 0,
            "緯度": 0,
            "経度": 0,
            "住居表示フラグ": 0,
            "代表フラグ": 0,
            "更新前履歴フラグ": 0,
            "更新後履歴フラグ": 0
        }
    }
    #csv.DictReaderで読込し、feature のテンプレートに適用後、FeatureCollectionにアペンド
    with open(csv_file_path, "r", encoding='shift_jis') as f:
        reader = csv.DictReader(f)
        fcnt = 0
        for row in reader:
            #feature のテンプレートに適用する処理
            feature = copy.deepcopy(feature_template)
            feature["id"] = fcnt
            feature["geometry"]["coordinates"] = [float(row["経度"]), float(row["緯度"])]
            feature["properties"]["OID"] = fcnt
            feature["properties"]["都道府県名"] = row["都道府県名"]
            feature["properties"]["市区町村名"] = row["市区町村名"]
            feature["properties"]["大字_丁目名"] = row["大字・丁目名"]
            feature["properties"]["小字_通称名"] = row["小字・通称名"]
            feature["properties"]["街区符号_地番"] = row["街区符号・地番"]
            feature["properties"]["座標系番号"] = int(row["座標系番号"])
            feature["properties"]["Ｘ座標"] = float(row["Ｘ座標"])
            feature["properties"]["Ｙ座標"] = float(row["Ｙ座標"])
            feature["properties"]["緯度"] = float(row["緯度"])
            feature["properties"]["経度"] = float(row["経度"])
            feature["properties"]["住居表示フラグ"] = int(row["住居表示フラグ"])
            feature["properties"]["代表フラグ"] = int(row["代表フラグ"])
            feature["properties"]["更新前履歴フラグ"] = int(row["更新前履歴フラグ"])
            feature["properties"]["更新後履歴フラグ"] = int(row["更新後履歴フラグ"])
            #FeatureCollectionにアペンド
            feature_collection["features"].append(feature)
            #次のレコード用にidをインクリメント
            fcnt +=1
    #FeatureCollectionをjsonとして書き出し保存
    with open(json_file_path, "w", encoding="utf_8") as f:
        json.dump(feature_collection, f, ensure_ascii=False, indent=4)
        
    return fcnt

### 大字町丁目の変換

In [2]:
csv_file_path1 = r"<YourDir>\位置情報参照データ\R5_東北地方\02000-17.0b\02000-17.0b\02_2023.csv"
geojson_file_path1 = r"<YourDir>\位置情報参照データ\R5_東北地方\大字町丁目_02_2023.json"
cnt1 = ooaza_cho_csv_to_geojson(csv_file_path1, geojson_file_path1)
print("convert {} features".format(cnt1))

convert 3145 features


### 街区の変換

In [3]:
csv_file_path2 = r"<YourDir>\位置情報参照データ\R5_東北地方\02000-22.0a\02000-22.0a\02_2023.csv"
geojson_file_path2 = r"<YourDir>\位置情報参照データ\R5_東北地方\街区_02_2023.json"
cnt2 = gaiku_csv_to_geojson(csv_file_path2, geojson_file_path2)
print("convert {} features".format(cnt2))

convert 186592 features
